In [2]:
setwd('/nfs/lab/projects/islet_cytok/analysis/socs1_figure')

In [2]:
library(scales)
library(RColorBrewer)
suppressPackageStartupMessages(library(plot3D))

Warning message:
“no DISPLAY variable so Tk is not available”


In [89]:
rec = read.csv("../selex/selex_variants_reclassified_annotated_with_atac_new.csv", row.names=1)

In [86]:
t1d = read.table('../selex/T1D.99credset_new.bed', stringsAsFactors = F)

In [94]:
finemap = subset(t1d, grepl("DEXI", t1d$V5))

In [95]:
finemap$pbsnp_diffatac_1 = finemap$V4 %in% rec$snpID[rec$pbs005 & rec$diff==T]

In [96]:
finemap$pbsnp_atac_1 = finemap$V4 %in% rec$snpID[rec$pbs005 & rec$atac==T]

In [97]:
finemap$pbsnp = finemap$V4 %in% rec$snpID[rec$pbs005 ]

In [98]:
table(finemap$pbsnp_diffatac_1,finemap$pbsnp_atac_1)

       
        FALSE TRUE
  FALSE    89    0
  TRUE      0    1

In [99]:
table(finemap$pbsnp)


FALSE  TRUE 
   84     6 

In [101]:
# bedtools intersect -a /nfs/lab/publicdata/1kg_snps/ALL.wgs.phase3_sites.bed.ids -b loci -wa > loci_snps.bed
# awk -v OFS="\t" '{print $5}' loci_snps.bed > loci_snps.rsid_list
# zgrep -F -f loci_snps.rsid_list /home/joshchiou/joshchiou-data2/T1D-GWAS/matched_cohorts/TOPMed/meta_analysis/T1D_meta.all_chr.hg19.sumstats.txt.gz > T1D.Chiou2020.select

In [102]:
gwas = read.table("T1D.Chiou2020.select", sep="\t", stringsAsFactors = F)
gwas$log10pval = -log10(gwas$V8)
gwas$credset = gwas$V1 %in% t1d$V4

In [103]:
ld = read.table("caqtl.ld.EUR.ld", header=T)

In [104]:
ld2 = read.table("caqtl.ld.EUR2.ld", header=T)

In [105]:
dim(ld)
dim(ld2)
dim(gwas)

[1] 560   7

[1] 406   7

[1] 17317    17

In [106]:
gwas = merge(gwas, ld[,6:7], by=1, all.x=T)

In [107]:
gwas$R2[gwas$V1 == '16:11194771:C:T']<-1
gwas$R2[is.na(gwas$R2)]<- 0

In [108]:
gwas = merge(gwas, ld2[,6:7], by=1, all.x=T)

In [109]:
gwas$R2.y[gwas$V1 == '11428643:C:G']<-1
gwas$R2.y[is.na(gwas$R2.y)]<- 0

In [110]:
my_palette = colorRampPalette( brewer.pal(9, "Blues")[3:9] )(20)
my_palette2 = colorRampPalette( brewer.pal(9, "Reds")[3:9] )(20)

color1   <- my_palette[as.numeric(cut(gwas$R2.x ,breaks = 20))]
color2  <- my_palette2[as.numeric(cut(gwas$R2.y ,breaks = 20))]

In [111]:
color = c(color1[1:head(which(gwas$V3 >11350000 ),1)],
          color2[head(which(gwas$V3 >11350000 ))[2]:length(color2)])

In [112]:
gwas$color = color
gwas$color[gwas$R2.y == 0 & gwas$R2.x==0]<-"gray"
gwas_toplot = gwas[order(gwas$log10pval),]

In [113]:
gwas_toplot = gwas_toplot[order(gwas_toplot$R2.x, decreasing = F),]
gwas_toplot = gwas_toplot[order(gwas_toplot$R2.y, decreasing = F),]

In [198]:
pdf("socs1_finemap.pdf")
par(mfrow=c(3,1), mar = c(4,2,6,4), las=1)
plot(log10pval~V3, gwas_toplot,  xlim= c(10934642,11913586), ylim=c(0,45),
     pch= 21,
     col = c(NA, alpha('white',0.5))[gwas_toplot$credset +1],
     #col=alpha('gray', 0.5),
     bg=alpha(gwas_toplot$color, 0.8)
     #col=alpha(c("gray", "red"),0.5)[gwas$credset +1]
     )
colkey(c("gray",my_palette), clim=c(0,1), clab="R2", add=T, side=4, length=0.5)
# plot(V7~V3, finemap, 
#   col=alpha(c("black", "red"),0.6)[finemap$pbsnp_diffatac_1 +1],
#      pch=16, xlim= c(10934642,11913586))

plot(V7~V3, finemap[finemap$V5 == 'DEXI_16:11194771:C:T',], 
     col=alpha(c("black", "red"),0.8)[finemap$pbsnp_diffatac_1[finemap$V5 == 'DEXI_16:11194771:C:T'] +1],
     pch=16, xlim= c(10934642,11913586))
 
plot(V7~V3, finemap[finemap$V5 == 'DEXI_16:11428643:C:G',], 
      col=alpha(c("black", "red"),0.8)[finemap$pbsnp_diffatac_1[finemap$V5 == 'DEXI_16:11428643:C:G'] +1],
   pch=16, xlim= c(10934642,11913586))

colkey(c("gray",my_palette2), clim=c(0,1), clab="R2", add=T, side=4, length=0.5)

    dev.off()


png 
  2

In [114]:
png("socs1_locus.png", width = 6.51, height = 1.064, units = 'in', res = 300)
par( mar=c(0,0,0,0))
plot(log10pval~V3, gwas_toplot,  xlim= c(10934642,11913586), ylim=c(0,45),
     pch= 21,axes=F,lwd=0.5, cex=0.6,
     col = c(NA, alpha('white',0.5))[gwas_toplot$credset +1],
     #col=alpha('gray', 0.5),
     bg=alpha(gwas_toplot$color, 0.8)
     #col=alpha(c("gray", "red"),0.5)[gwas$credset +1]
     )
dev.off()

png 
  2

In [115]:
png("socs1_locus_norbord.png", width = 6.51, height = 1.064, units = 'in', res = 300)
par( mar=c(0,0,0,0))
plot(log10pval~V3, gwas_toplot,  xlim= c(10934642,11913586), ylim=c(0,45),
     pch= 19,axes=F,cex=0.6,
     col=alpha(gwas_toplot$color, 0.8)
     
     )
dev.off()

png 
  2

In [132]:
write.table(finemap[finemap$V5 == 'DEXI_16:11428643:C:G',1:3], "finemap_snps.bed",
           sep="\t", col.names=F, row.names=F, quote=F)

## APOTRACKER experiment

In [3]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(RColorBrewer))
suppressPackageStartupMessages(library(gplots))
suppressPackageStartupMessages(library(beeswarm))
suppressPackageStartupMessages(library(tidyr))

In [5]:
new = read.table("endoc_ko.txt", row.names=1)

In [19]:
df = data.frame(Treatment = c(rep('Control', 8), rep('Cytokine', 8)), 
                shRNA = rep(c(rep('Scramble', 4), rep('SOCS1', 4)),2 ) ,
          meas = c(t(new[1,]), t(new[2,])))

In [22]:
df

Treatment,shRNA,meas
<chr>,<chr>,<dbl>
Control,Scramble,2.12
Control,Scramble,2.44
Control,Scramble,2.30
Control,Scramble,1.30
Control,SOCS1,3.78
Control,SOCS1,2.09
Control,SOCS1,0.58
Control,SOCS1,1.73
Cytokine,Scramble,3.02


In [29]:
anova(lm(meas ~ shRNA , df[df$Treatment=='Control',]))

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
shRNA,1,0.00005,0.00005,4.967463e-05,0.994605
Residuals,6,6.03930,1.00655,NA,NA


In [68]:
anova(lm(meas ~ shRNA , df[df$Treatment=='Cytokine',]))

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
shRNA,1,282.8631,282.86311,7.648343,0.03261316
Residuals,6,221.9015,36.98358,NA,NA


In [71]:
res.aov = aov(meas ~ shRNA*Treatment , df)

In [73]:
summary(res.aov)

                Df Sum Sq Mean Sq F value  Pr(>F)   
shRNA            1  141.6   141.6   7.452 0.01827 * 
Treatment        1  199.0   199.0  10.478 0.00713 **
shRNA:Treatment  1  141.3   141.3   7.439 0.01835 * 
Residuals       12  227.9    19.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [76]:
TukeyHSD(res.aov)

  Tukey multiple comparisons of means
    95% family-wise confidence level

Fit: aov(formula = meas ~ shRNA * Treatment, data = df)

$shRNA
                  diff      lwr      upr     p adj
SOCS1-Scramble 5.94875 1.200754 10.69675 0.0182723

$Treatment
                    diff      lwr      upr     p adj
Cytokine-Control 7.05375 2.305754 11.80175 0.0071275

$`shRNA:Treatment`
                                      diff       lwr       upr     p adj
SOCS1:Control-Scramble:Control      0.0050 -9.144579  9.154579 1.0000000
Scramble:Cytokine-Scramble:Control  1.1100 -8.039579 10.259579 0.9832324
SOCS1:Cytokine-Scramble:Control    13.0025  3.852921 22.152079 0.0056562
Scramble:Cytokine-SOCS1:Control     1.1050 -8.044579 10.254579 0.9834491
SOCS1:Cytokine-SOCS1:Control       12.9975  3.847921 22.147079 0.0056720
SOCS1:Cytokine-Scramble:Cytokine   11.8925  2.742921 21.042079 0.0105609
